In [45]:
pip install -U webdriver_manager

Note: you may need to restart the kernel to use updated packages.


In [47]:
pip install -U selenium

Note: you may need to restart the kernel to use updated packages.


In [49]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By # used to import different ways to access data in the XML or HTML file
from selenium.webdriver.chrome.service import Service # no longer need to download a driver file, use service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException

from webdriver_manager.chrome import ChromeDriverManager # used to manage the Chrome driver to emulate a Chrome web browser

import time
import random

In [55]:
browser = webdriver.Chrome()
browser.maximize_window()

# Empty lists for data
state_names = []
county_names = []
trump_votes = []
clinton_votes = []

# Visit the main URL
url = "https://www.nytimes.com/elections/2016/results/president"
browser.get(url)
time.sleep(2)

# Find all links inside the footer (you may need to adjust this based on the actual HTML structure)
state_links = browser.find_elements(By.XPATH, '//ul[@class="eln-page-list"]/li/a')

# Get each state url and store in empty list address
state_urls = [link.get_attribute('href') for link in state_links]
address = []
for url in state_urls:
    address.append(url)

#Only scraping Alabama and Arizona state pages
for link in [address[0], address[2],address[3],address[4],address[5],address[6],address[7],address[9],address[10],address[11],address[12],address[13],address[14],address[15],address[16],address[17],address[18],address[19],address[20],address[21],address[22],address[23],address[24],address[25],address[26],address[27],address[28],address[29],address[30],address[31],address[32],address[33],address[34],address[35],address[36],address[37],address[38],address[39],address[40],address[41],address[42],address[43],address[44],address[45],address[46],address[47],address[48],address[49],address[50]]:
    browser.get(link)
    time.sleep(random.uniform(1, 3))

    #Extracting the state name from the state url
    state_name = link.split("/")[-1]

    #Finding the table with county level table
    tables = browser.find_elements(By.XPATH, '//table[@class="eln-county-table"]')
    
    #Clicking the show all counties button to expand the table view
    try:
        show_all_button = browser.find_element(By.CLASS_NAME, "eln-toggle-counties")
        show_all_button.click()
        time.sleep(1)
    except:
        pass

    #Locating the county level table on each state link
    for table in tables:
        headers = table.find_elements(By.TAG_NAME, 'th')
        header_texts = [h.text.strip().lower() for h in headers]
        
        #Finding only the presidental election results table
        if 'trump' in header_texts and 'clinton' in header_texts:

            #Finding each instance in a table
            state_rows = table.find_elements(By.TAG_NAME, 'tr')[1:]

            #Extracting the cell data for each row, county name, trump votes, clinton votes
            for row in state_rows:
                cells = row.find_elements(By.TAG_NAME, "td")
                if len(cells) >= 3:
                    county_names.append(cells[0].text)
                    trump_votes.append(cells[1].text)
                    clinton_votes.append(cells[2].text)
                    state_names.append(state_name)

#Saving the extracted data to a dataframe
election_results = pd.DataFrame({
    "state": state_names,
    "county": county_names,
    "trump votes": trump_votes,
    "clinton votes": clinton_votes
})
    
    
# Closing the WebDriver
browser.quit()

In [57]:
#Displaying the data frame to verify web scraping 
election_results

,state,county,trump votes,clinton votes
0,alabama,Jefferson,"134,768","156,873"
1,alabama,Mobile,"95,116","72,186"
2,alabama,Madison,"89,520","62,822"
3,alabama,Shelby,"73,020","22,977"
4,alabama,Montgomery,"34,003","58,916"
...,...,...,...,...
4579,wyoming,Crook,"3,348",273
4580,wyoming,Washakie,"2,911",532
4581,wyoming,Weston,"3,033",299
4582,wyoming,Hot Springs,"1,939",400


In [59]:
#Checking data types of the data frame
election_results.dtypes

state            object
county           object
trump votes      object
clinton votes    object
dtype: object

In [61]:
#Changing the data type of votes from object to integar 
election_results['trump votes'] = election_results['trump votes'].str.replace(',', '').astype(int)
election_results['clinton votes'] = election_results['clinton votes'].str.replace(',', '').astype(int)

#Changning state and county data type from object to category
election_results["state"] = election_results["state"].astype("category")
election_results["county"] = election_results["county"].astype("category")

In [63]:
#Rechecking data types to verify conversion 
election_results.dtypes

state            category
county           category
trump votes         int64
clinton votes       int64
dtype: object

In [65]:
#Creating a dervived column for total votes and vote percentage 
election_results["total votes"] = election_results["trump votes"] + election_results["clinton votes"]
election_results["trump vote %"] = (election_results["trump votes"] / election_results["total votes"] * 100).round(2)
election_results["clinton vote %"] = (election_results["clinton votes"] / election_results["total votes"] * 100).round(2)

In [67]:
election_results

,state,county,trump votes,clinton votes,total votes,trump vote %,clinton vote %
0,alabama,Jefferson,134768,156873,291641,46.21,53.79
1,alabama,Mobile,95116,72186,167302,56.85,43.15
2,alabama,Madison,89520,62822,152342,58.76,41.24
3,alabama,Shelby,73020,22977,95997,76.06,23.94
4,alabama,Montgomery,34003,58916,92919,36.59,63.41
...,...,...,...,...,...,...,...
4579,wyoming,Crook,3348,273,3621,92.46,7.54
4580,wyoming,Washakie,2911,532,3443,84.55,15.45
4581,wyoming,Weston,3033,299,3332,91.03,8.97
4582,wyoming,Hot Springs,1939,400,2339,82.90,17.10


In [69]:
#Checking data type of the dervived columns
election_results.dtypes

state             category
county            category
trump votes          int64
clinton votes        int64
total votes          int64
trump vote %       float64
clinton vote %     float64
dtype: object

In [71]:
#Checking for duplicates
duplicate_rows = election_results.duplicated()
print(f"Duplicates: {duplicate_rows.sum()}")

Duplicates: 0


In [73]:
#Saving the data frame to a csv file
election_results.to_csv("election_results_complete.csv", index=False,encoding='utf-8')